In [ ]:
import pandas as pd
import numpy as np
import featuretools as ft
from datetime import datetime
from dateutil.relativedelta import relativedelta


In [ ]:
#https://github.com/IBM/telco-customer-churn-on-icp4d
URL_TO_DATA = 'https://raw.githubusercontent.com/IBM/telco-customer-churn-on-icp4d/master/data/Telco-Customer-Churn.csv'

CUSTOMERS = ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents']
SUBSCRIPTIONS = ['customerID', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
 
BILLING = [ 'customerID', 'tenure', 'Contract', 
 'PaperlessBilling',
 'PaymentMethod',
 'MonthlyCharges',
 'TotalCharges',
 'Churn']

BACK_COUNT_DATE = datetime.datetime(2022,1,1)



- ft titanic https://www.kaggle.com/code/liananapalkova/automated-feature-engineering-for-titanic-dataset/notebook
- the dataset https://github.com/IBM/telco-customer-churn-on-icp4d/blob/master/data/Telco-Customer-Churn.csv




### Denormalize into 
- customer_df:      customer_id (PK), subscription_id (FK), gender, SeniorCitizen, Partner, Dependents, tenure, Churn
- services_df:      service_id (PK), service_name
- subscription_df:  subscription_id (PK), service_id, customer_id
- billing_df:       billing_id, Contract, PaperlessBilling, PaymentMethod, MonthlyCharges, TotalCharges


In [ ]:
# in case of CERTIFICATE_VERIFY_FAILED run Install Certificates.command
# see also https://stackoverflow.com/questions/50236117/scraping-ssl-certificate-verify-failed-error-for-http-en-wikipedia-org
df = pd.read_csv(filepath_or_buffer=URL_TO_DATA, index_col=0)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe(include="all")

In [ ]:
# convert
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

# generate syntatic time index 
df['contract_start_date'] = list(map(lambda tenure, dat = BACK_COUNT_DATE: dat - relativedelta(months = -8), df['tenure']))


In [ ]:
df_customers = df[CUSTOMERS].copy()
df_subscriptions = df[SUBSCRIPTIONS].copy()
df_billing = df[BILLING].copy()


In [ ]:
dataframes = {
    "customers": (df_customers, "customer_id"),
    "subscriptions": (df_subscriptions, "customer_id", "customer_id"),
    "billing": (df_billing, "customer_id", "customer_id"),
}


relationships = [
    ("customers", "customer_id", "subscriptions", "customer_id"),
    ("subscriptions", "customer_id", "billing", "customer_id"),
]

feature_matrix_customers, features_defs = ft.dfs(
    dataframes=dataframes,
    relationships=relationships,
    target_dataframe_name="customers",
)
feature_matrix_customers

In [ ]:
entity_set = ft.EntitySet(id="customer_churn_entities")

# https://featuretools.alteryx.com/en/stable/generated/featuretools.EntitySet.add_dataframe.html#featuretools.EntitySet.add_dataframe
entity_set.add_dataframe(
    dataframe_name="customer_churn", index="customerID", dataframe=df
)

In [ ]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

import datetime

first_day_of_the_month = datetime.date.today().replace(day=1)
first_day_of_the_month

In [ ]:
df['contract_start_date'] = list(map(lambda tenure, dat = BACK_COUNT_DATE: dat - relativedelta(months = -8), df['tenure']))



#datetime.date.today().replace(day=1) - relativedelta(months = -8)

In [ ]:
df['contract_start_date']